In [2]:
# https://docs.google.com/document/d/1e9oakyEftdHp4zGz2F0WP-_X7GEzEKYGaPvu1SguuCE/edit

import pandas as pd
import numpy as np
import matplotlib as plt

# https://pypi.org/project/yfinance/#description
import yfinance as yf

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [3]:
portfolio = ['MSFT', 'AAPL', 'IBM', 'WMT', 'SHOP']
df = yf.download(portfolio,'2022-1-1') # Pulling the daily closing stock price for the aforementioned stocks from the start of 2022-onwards.

[*********************100%***********************]  5 of 5 completed


In [4]:
print(df.tail())

             Adj Close                                                       Close              \
                  AAPL         IBM        MSFT        SHOP         WMT        AAPL         IBM   
Date                                                                                             
2022-02-17  168.880005  124.970001  290.730011  660.000000  138.880005  168.880005  124.970001   
2022-02-18  167.300003  124.349998  287.929993  656.880005  137.990005  167.300003  124.349998   
2022-02-22  164.320007  123.919998  287.720001  626.159973  136.449997  164.320007  123.919998   
2022-02-23  160.070007  122.070000  280.269989  626.609985  135.050003  160.070007  122.070000   
2022-02-24  155.929993  120.089996  278.489197  611.114990  133.029999  155.929993  120.089996   

                                                      High                                      \
                  MSFT        SHOP         WMT        AAPL         IBM        MSFT        SHOP   
Date               

### Portfolio of 5 stocks is now present. Data is daily changes, from the start of the year, and contains 6 features.

# Transformations:

### Transform 1: Group by industry

Calculate:
* Market value per industry (dollar amount) for the day
* % change of market value from previous day for each industry

(market value = adjusted close * volume)

In [ ]:
def aggregate_stock(ticker):
    

In [18]:
a = yf.Ticker("MSFT").info

In [21]:
a['sector']

'Technology'

In [40]:
def pull_data(tickers):
    df = yf.download(tickers,'2022-2-1')
    # for ticker in tickers:
    #     sector = yf.Ticker(ticker).info['sector']
        
    print(df.tail())
    
pull_data(portfolio)

[*********************100%***********************]  5 of 5 completed
             Adj Close                                                       Close              \
                  AAPL         IBM        MSFT        SHOP         WMT        AAPL         IBM   
Date                                                                                             
2022-02-17  168.880005  124.970001  290.730011  660.000000  138.880005  168.880005  124.970001   
2022-02-18  167.300003  124.349998  287.929993  656.880005  137.990005  167.300003  124.349998   
2022-02-22  164.320007  123.919998  287.720001  626.159973  136.449997  164.320007  123.919998   
2022-02-23  160.070007  122.070000  280.269989  626.609985  135.050003  160.070007  122.070000   
2022-02-24  158.229996  119.919800  285.480011  649.359985  133.330002  158.229996  119.919800   

                                                      High                                      \
                  MSFT        SHOP         WMT  

In [85]:
df = yf.download(portfolio,'2022-2-1', group_by='Ticker')
df = df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1) # Trenton McKinney, https://stackoverflow.com/questions/63107594/how-to-deal-with-multi-level-column-names-downloaded-with-yfinance/63107801#63107801

[*********************100%***********************]  5 of 5 completed


In [86]:
df

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2022-02-01,AAPL,174.387817,174.610001,174.839996,172.309998,174.009995,86213900
2022-02-01,IBM,133.916901,135.529999,135.960007,132.500000,133.759995,6206400
2022-02-01,MSFT,308.122894,308.760010,310.630005,305.130005,310.410004,40950400
2022-02-01,SHOP,984.200012,984.200012,988.500000,937.851013,984.299988,1752100
2022-02-01,WMT,140.910004,140.910004,141.149994,138.990005,139.210007,6774400
...,...,...,...,...,...,...,...
2022-02-24,AAPL,157.860001,157.860001,158.679993,152.000000,152.580002,67915733
2022-02-24,IBM,119.790001,119.790001,120.680000,118.809998,120.000000,2456964
2022-02-24,MSFT,284.510010,284.510010,286.470001,271.519989,272.510010,22905435


### Pull industry for each ticker (for eventual aggregation)

In [84]:
portfolio = ['MSFT', 'AAPL', 'IBM', 'WMT', 'SHOP']
portfolio_df = pd.DataFrame(portfolio).rename(columns={0: 'Ticker'})
portfolio_df['Industry'] = portfolio_df.Ticker.apply(lambda x: yf.Ticker(x).info['sector'])
portfolio_df

,Ticker,Industry
0,MSFT,Technology
1,AAPL,Technology
2,IBM,Technology
3,WMT,Consumer Defensive
4,SHOP,Technology


### Transform 1: % change of previous day close price vs current day close price

In [8]:
# Below we'll calculate the percent change of Adj Close price for each stock and, if the price change for one exceeds a predermined threshold, an email notification will be sent out.
last_two_days = df['Adj Close'].iloc[-2:].pct_change()
percent_change = last_two_days.iloc[-1]
percent_change

AAPL   -0.025864
IBM    -0.016220
MSFT   -0.006354
SHOP   -0.024728
WMT    -0.014957
Name: 2022-02-24 00:00:00, dtype: float64

### Transform 1: % change of previous day close price vs current day close price